# PARAMETRIC VAR Calculation for 5 Stock Portfolio

In [1]:
# importing relevant library for VAR calculation
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt

In [2]:
# setting years, start date, end date for fetching data from the yfinance library
years = 15
enddate = dt.datetime.now()
startdate = enddate - dt.timedelta(days = 365*years)

In [3]:
# The ticker contains indices
# fetching the data through yfinance library 
# we are only going to take adjusted closing price
tickers = ['SPY','BND','GLD','QQQ','VTI']
data = yf.download(tickers, start = startdate, end = enddate)['Close']
data

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  5 of 5 completed


Ticker,BND,GLD,QQQ,SPY,VTI
Date,,,,,
2010-04-22,51.698231,111.839996,43.925358,91.965431,47.387188
2010-04-23,51.659161,113.190002,44.108692,92.565750,47.707932
2010-04-26,51.633125,112.750000,44.012669,92.216194,47.539921
2010-04-27,51.821907,114.629997,43.078461,90.035210,46.394379
2010-04-28,51.789368,114.309998,43.104671,90.719147,46.699852
...,...,...,...,...,...
2025-04-11,72.019997,297.929993,454.399994,533.940002,261.739990
2025-04-14,72.430000,296.230011,457.480011,539.119995,264.149994
2025-04-15,72.610001,297.779999,457.989990,537.609985,263.690002


In [4]:
# getting the daily return of the 5 stocks
log_ret = np.log(data/data.shift(1))
log_ret = log_ret.dropna()
log_ret

Ticker,BND,GLD,QQQ,SPY,VTI
Date,,,,,
2010-04-23,-0.000756,0.011999,0.004165,0.006506,0.006746
2010-04-26,-0.000504,-0.003895,-0.002179,-0.003783,-0.003528
2010-04-27,0.003650,0.016537,-0.021454,-0.023935,-0.024391
2010-04-28,-0.000628,-0.002795,0.000608,0.007568,0.006563
2010-04-29,0.000377,-0.000262,0.017270,0.012321,0.012998
...,...,...,...,...,...
2025-04-11,-0.001110,0.018907,0.018255,0.017686,0.016604
2025-04-14,0.005677,-0.005722,0.006755,0.009655,0.009165
2025-04-15,0.002482,0.005219,0.001114,-0.002805,-0.001743


In [5]:
# lets assume that we have 1 million dollar portfolio and we are equally investing in the 5 stocks
port_val = 1000000
weights = np.array([1/len(tickers)]*len(tickers))
weights

array([0.2, 0.2, 0.2, 0.2, 0.2])

In [6]:
# Multiplying the indices return with its designated weights and summing it up
hist_return = (log_ret* weights).sum(axis = 1)
hist_return

Date
2010-04-23    0.005732
2010-04-26   -0.002778
2010-04-27   -0.009919
2010-04-28    0.002263
2010-04-29    0.008541
                ...   
2025-04-11    0.014068
2025-04-14    0.005106
2025-04-15    0.000853
2025-04-16   -0.007865
2025-04-17   -0.000653
Length: 3771, dtype: float64

In [7]:
# calculating the covariances of 2 indices and presenting it in a convariance matrix and also annualising the same
cov_mat = log_ret.cov()*252
cov_mat

Ticker,BND,GLD,QQQ,SPY,VTI
Ticker,,,,,
BND,0.002397,0.002291,0.000272,0.000073,0.000122
GLD,0.002291,0.024117,0.001814,0.001431,0.001567
QQQ,0.000272,0.001814,0.043936,0.034179,0.034837
SPY,0.000073,0.001431,0.034179,0.030629,0.031213
VTI,0.000122,0.001567,0.034837,0.031213,0.032056


In [8]:
# calculating the standard deviation of the 5 stock portfolio
# here @ calculate the dot matrix
std_port = np.sqrt(weights.T @ cov_mat @ weights)
std_port

0.11810715175195349

In [9]:
# an array of confidence interval i.e., 90%,95% and 99%
# getting scipy library to get norm.ppf which help in getting z-score for the given confidence interval 
conf_level = [0.90,0.95,0.99]
from scipy.stats import norm
VARs=[]

In [10]:
# calculating the annualised average return of the portfolio 
hist_return.mean()*252

0.09493100999421002

In [13]:
# using for loop to get VAR for 90%, 95%, and 99% confidence interval
# The formula = (P.return - (P.std * z-score)) * P.value
for cl in conf_level:
    VAR = port_val* (hist_return.mean()*252 - (std_port * norm.ppf(cl)))
    print(VAR)

-56429.39523551967
-99337.96693389861
-179827.311392966
